In [3]:
# IMPORTANDO AS BIBLIOTECAS

import pandas as pd
import psycopg2
import email
import openpyxl
import os
from sqlalchemy import create_engine        

pd.set_option('display.max_columns',100)

In [ ]:
#EXTRAINDO AS PLANILHAS
df = pd.read_excel('GARE 01.04_teste.xlsx')

# DICIONARIO
# 101 = PESSOA FISICA
# 202 = SOCIEDADE LIMITADA, ECONOMIA MISTA, ANÔNIMA, POR COTAS, ETC
# 203 = FUNDO DE INVESTIMENTO IMOBILIÁRIO
# 501 = FUNDO DE INVESTIMENTO (EXCETO FUNDO DE INVESTIMENTO IMOBILIÁRIO)
# 1002 = INVESTIDOR NÃO-RESIDENTE RESOLUÇÃO N 2.689 - PJ
# 901 = TEMPLO DE QUALQUER CULTO
# 1001 = INVESTIDOR NÃO-RESIDENTE RESOLUÇÃO N 2.689 - PF
# 405 = SOCIEDADE CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS E CÂMBIO
# 201 =  
# 402 = BANCO DE INVESTIMENTOS
# 601 = FUNDO DE PREVIDÊNCIA PRIVADA - ABERTO
# 904 = INSTITUIÇÃO DE EDUCAÇÃO E ASSITÊNCIA SOCIAL, SEM FINS LUCRATIVOS

NameError: name 'pd' is not defined

In [ ]:
# VARIAVEL PARA RENOMEAR AS COLUNAS
cols_rename = {'CPF/CNPJ':'DOCUMENTO', 'Quantidade':'TOTAL', 'Tipo':'Classe_BVMF', 'Nome':'NOME'}

#VARIAVEL PARA EXCLUIR AS COLUNAS
cols_drop = ['ISIN', 'Fundo', 'Data', 'Data Posição']


# FUNÇÃO PARA CATEGORIZAR PF E PJ
def identificadorid(x):
    if x == 101 or x == 1001:
        return 'PF'
    else:
        return 'PJ'

# FUNÇÃO PARA CRIAR A COLUNA DE Classe_BVMF
def categoria(ct):
    if ct == 101:
        return 'PESSOA FISICA'
    elif ct == 202:
        return 'SOCIEDADE LIMITADA, ECONOMIA MISTA, ANÔNIMA, POR COTAS, ETC'
    elif ct == 203:
        return 'FUNDO DE INVESTIMENTO IMOBILIÁRIO'
    elif ct == 501:
        return 'FUNDO DE INVESTIMENTO (EXCETO FUNDO DE INVESTIMENTO IMOBILIÁRIO)'
    elif ct == 1002:
        return 'INVESTIDOR NÃO-RESIDENTE RESOLUÇÃO N 2.689 - PJ'
    elif ct == 901:
        return 'TEMPLO DE QUALQUER CULTO'
    elif ct == 1001:
        return 'INVESTIDOR NÃO-RESIDENTE RESOLUÇÃO N 2.689 - PF'
    elif ct == 405:
        return 'SOCIEDADE CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS E CÂMBIO'
    elif ct == 201:
        return 'SOCIEDADE LIMITADA, ECONOMIA MISTA, ANÔNIMA, POR COTAS, ETC'
    elif ct == 402:
        return 'BANCO DE INVESTIMENTOS'
    elif ct == 601:
        return 'FUNDO DE PREVIDÊNCIA PRIVADA - ABERTO'
    elif ct == 904:
        return 'INSTITUIÇÃO DE EDUCAÇÃO E ASSITÊNCIA SOCIAL, SEM FINS LUCRATIVOS'
    else:
        return 'ERRO DE CLASSIFICAÇÃO'

#FUNÇÃO PARA TRATAR OS CPFS COM NUMERO FALTANDO NA COLUNA DE ID_INVESTIDOR
def tratar_doc(TP, doc):
    
    doc = str(doc)
    doc = doc.split('.')[0]

    if TP == 'PF':
        doc = doc.zfill(11)
        return doc
    else:
        doc = doc.zfill(14)[:14]
        return doc

#def excluir_5digitos(cpf_cnpj):
    return str(cpf_cnpj)[5:]    


# FUNÇÃO PARA TRATAR A TABELA DE MANEIRA GERAL
def tratar_colunas(x):
    #RENOMEIA
    x = x.rename(columns = cols_rename)
    #CRIA A COLUNA DE PF E PJ
    x['PF_PJ'] = x.Classe_BVMF.apply(identificadorid)
    #MAPEANDO A COLUNA DE TIPO
    x['Classe_BVMF'] = x.Classe_BVMF.apply(categoria)
    #MOVENDO AS COLUNAS PARA FORMATAR A TABELA DE MANEIRA CORRETA
    x = x[['DOCUMENTO', 'NOME', 'PF_PJ', 'Classe_BVMF', 'TOTAL']]
    #RETIRA OS ESPAÇOS DO NOME
    x['NOME'] = x['NOME'].str.strip()
    #APLICANDO A FUNÇÃO PARA FORMATAR O CPF E CNPJ NA COLUNA ID_Investidor
    x['DOCUMENTO'] = x['DOCUMENTO'].astype(str)
    #APLICANDO A FUNÇÃO PARA FORMATAR O CPF E CNPJ NA COLUNA ID_Investidor 
    x['DOCUMENTO'] = x.apply(lambda row: tratar_doc(row['PF_PJ'], row['DOCUMENTO']), axis = 1)
    #EXCLUINDO OS 5 PRIMEIROS DIGITOS
    #x['DOCUMENTO'] = x.DOCUMENTO.apply(excluir_5digitos)
    #RETIRA OS ESPAÇOS DO ID_Investidor
    x['DOCUMENTO'] = x['DOCUMENTO'].str.strip()
    #CRIANDO A CHAVE DE LIGAÇÃO
    x['CHAVE'] = x['DOCUMENTO'] + x['NOME'].str[:30]
    #ORDENANDO AS COLUNAS
    x = x[['CHAVE', 'DOCUMENTO', 'NOME', 'PF_PJ', 'TOTAL']]
    
    

    return x

In [ ]:
# TRATANDO MEU DATAFRAME
df = tratar_colunas(df)

df

In [ ]:
df_agrupado = df.groupby(['CHAVE']).size().reset_index(name='Quantidade')

# Exibir as linhas onde há mais de uma ocorrência (ou seja, duplicados)
df_duplicados = df_agrupado[df_agrupado['Quantidade'] > 1]

# Exibir o resultado
print(df_duplicados)

In [ ]:
df = df.groupby(['DOCUMENTO'], as_index=False).agg({'CHAVE':'first',
                                                       'DOCUMENTO':'first',
                                                       'NOME':'first',
                                                       'PF_PJ':'first',
                                                       'TOTAL':'sum'})
df

In [ ]:
#SALVANDO O DATAFRAME EM .XLSX EM UMA PASTA ESPECIFICA

#MAPEANDO O CAMINHO E SALVANDO A PLANILHA TRATADA COMO .XLSX
pasta_destino = r'C:\Users\LucasCavalcante\Desktop\PASSIVO\RELATÓRIOS_FORMATADOS\ARQUIVOS_B3\GARE\ABR'
#CASO A PASTA NÃO EXISTA UMA SERA CRIADA IMEDIANTAMENT
os.makedirs(pasta_destino, exist_ok=True)
#INSERE EM UMA VARIAVEL O CAMINHO E O NOME DO ARQUIVO
caminho_arquivo_excel = os.path.join(pasta_destino, '01.04_teste.xlsx')
#EXPORTA O ARQUIVO PARA .XLSX E O SALVA NA PASTA QUE O CAMINHO INDICA
df.to_excel(caminho_arquivo_excel, index=False)

print('Arquivo salvo com sucesso')